# Starting Spark Session

In [1]:
from pyspark.sql import SparkSession

# Spark session & context
spark = (SparkSession
         .builder
         .master("local")
         .appName("read-postgres")
         # Add postgres jar
         .config("spark.driver.extraClassPath", "/home/jovyan/work/jars/postgresql-42.4.0.jar")
         .getOrCreate())
sc = spark.sparkContext

## Read Postgres

In [2]:
df_movies = (
    spark.read
    .format("jdbc")
    .option("url", "jdbc:postgresql://mypostgres:5432/test")
    .option("dbtable", "public.movies")
    .option("user", "airflow")
    .option("password", "airflow")
    .load()
)

Py4JJavaError: An error occurred while calling o33.load.
: org.postgresql.util.PSQLException: ERROR: relation "public.movies" does not exist
  Position: 15
	at org.postgresql.core.v3.QueryExecutorImpl.receiveErrorResponse(QueryExecutorImpl.java:2676)
	at org.postgresql.core.v3.QueryExecutorImpl.processResults(QueryExecutorImpl.java:2366)
	at org.postgresql.core.v3.QueryExecutorImpl.execute(QueryExecutorImpl.java:356)
	at org.postgresql.jdbc.PgStatement.executeInternal(PgStatement.java:490)
	at org.postgresql.jdbc.PgStatement.execute(PgStatement.java:408)
	at org.postgresql.jdbc.PgPreparedStatement.executeWithFlags(PgPreparedStatement.java:181)
	at org.postgresql.jdbc.PgPreparedStatement.executeQuery(PgPreparedStatement.java:133)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCRDD$.getQueryOutputSchema(JDBCRDD.scala:68)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCRDD$.resolveTable(JDBCRDD.scala:58)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCRelation$.get

In [3]:
df_ratings = (
    spark.read
    .format("jdbc")
    .option("url", "jdbc:postgresql://mypostgres:5432/test")
    .option("dbtable", "public.ratings")
    .option("user", "airflow")
    .option("password", "airflow")
    .load()
)

# Top 10 movies with more ratings

In [4]:
df_movies = df_movies.alias("m")
df_ratings = df_ratings.alias("r")

df_join = df_ratings.join(df_movies, df_ratings.movieId == df_movies.movieId).select("r.*","m.title")

In [5]:
from pyspark.sql import functions as F

df_result = (
    df_join
    .groupBy("title")
    .agg(
        F.count("timestamp").alias("qty_ratings")
        ,F.mean("rating").alias("avg_rating")
    )
    .sort(F.desc("qty_ratings"))
    .limit(10)
)

In [7]:
df_result.coalesce(1).write.format("csv").mode("overwrite").save("/home/jovyan/work/resources/data/output_postgres", header=True)